In [ ]:
from pprint import pprint
import numpy as np
import itertools


In [ ]:
def tiling(n_rows=1, n_cols=1, dx=0, dy=0):
    """Generator for tile indices

    Parameters
    ----------
    n_rows : int, optional
        Number of rows, by default 1
    n_cols : int, optional
        Number of columns, by default 1

    Returns
    -------
    list
        List of tuples of tile positions
    """
    
    # moves = [{"type": "move_stage", "dx": j*dx, "dy": i*dy} for i in range(n_rows) for j in range(n_cols)]
    moves = [[j*dx, i*dy] for i in range(n_rows) for j in range(n_cols)]

    return moves

def update_stage_moves(move_list, initial_position=(0, 0, 0)):
    for move in move_list:
        move[0] += initial_position[0]
        move[1] += initial_position[1]
    return move_list

def volume(n_slices, step_size):
    if n_slices % 2 == 0:
        n_slices += 1
        print("Must be odd atm, adding 1")

    list_ = list(np.linspace(-(n_slices-1)//2, n_slices//2, n_slices)*step_size)
    return list_[::-1]

def update_objective_moves(move_list, initial_position_obj=0):
    # add initial_position[2] to each value in the list: move_list
    for i, move in enumerate(move_list):
        move_list[i] += initial_position_obj    
    return move_list

def workflow_generator(initial_position, tile_list: list[float] = None, volume_list: list[float]=None):
    if tile_list is None and volume_list is None:
        raise ValueError("Must provide either tile_list or volume_list")
    moves = [[tile_move[0], tile_move[1], volume_move] for tile_move in tile_list for volume_move in volume_list]
    moves.append(initial_position)
    # moves = list(itertools.chain.from_iterable([move, ["image"]] for move in moves))
    return moves


In [ ]:
tile_moves = tiling(2, 2, 1, 1)
pprint(tile_moves)
volume_moves = volume(3, 1)
pprint(volume_moves)


In [ ]:
initial_position = [0, 0, 0]

In [ ]:
updated_tile_moves = update_stage_moves(tile_moves, initial_position)
pprint(updated_tile_moves)
updated_volume_moves = update_objective_moves(volume_moves, initial_position_obj=initial_position[2])
pprint(updated_volume_moves)

In [ ]:
move_list = workflow_generator(initial_position, tile_list=updated_tile_moves, volume_list=updated_volume_moves)
pprint(move_list)


In [ ]:
# check the difference in each element of the list compared to the previous element
# if the difference is 0, then the stage is not moving
from copy import deepcopy



current_position = initial_position

relative_moves = []
images_taken = []

for move in updated_tile_moves[1:]:
    for v_move in updated_volume_moves:
        # print(f"Objective moves to: {v_move}")
        # print(f"Objective moves by {v_move - current_position[2]}")
        relative_moves.append({"type": "move_objective", "dz": v_move - current_position[2]})
        current_position[2] = v_move
        # print(f"Current position: {current_position}")
        print(f"Image taken at: {current_position}")
        relative_moves.append({"type": "image", "sync": None, "settings": None})
        images_taken.append(deepcopy(current_position))
    # print(f"Stage moves to: {move[0], move[1]}")
    # print(f"Stage moves by {move[0] - current_position[0], move[1] - current_position[1]}")
    relative_moves.append({"type": "move_stage", "dx": move[0] - current_position[0], "dy": move[1] - current_position[1]})
    current_position[0] = move[0]
    current_position[1] = move[1]
    # print(f"Current position: {current_position}")

for v_move in updated_volume_moves:
        # print(f"Objective moves to: {v_move}")
        # print(f"Objective moves by {v_move - current_position[2]}")
        relative_moves.append({"type": "move_objective", "dz": v_move - current_position[2]})
        current_position[2] = v_move
        # print(f"Current position: {current_position}")
        print(f"Image taken at: {current_position}")
        relative_moves.append({"type": "image", "sync": None, "settings": None})
        images_taken.append(deepcopy(current_position))



In [ ]:
pprint(relative_moves)

In [ ]:
pprint(images_taken)